In [45]:
# Subset: poetry

import pandas as pd
import numpy as np

In [50]:
df = pd.read_json("goodreads_interactions_poetry.json", lines=True)

In [51]:
df.head(2)

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,1384,1bad0122cebb4aa9213f9fe1aa281f66,True,4,,Wed May 09 09:33:44 -0700 2007,Wed May 09 09:33:44 -0700 2007,,
1,8842281e1d1347389f2ab93d60773d4d,1376,eb6e502d0c04d57b43a5a02c21b64ab4,True,4,,Wed May 09 09:33:18 -0700 2007,Wed May 09 09:33:18 -0700 2007,,


In [66]:
# 1. discard interactions of users with less than 10 interactions
counts = df['user_id'].value_counts()
df = df[df['user_id'].isin(counts[counts >=10].index)]

In [68]:
df.shape

(1848958, 10)

In [79]:
# 2. get unique user_ids
ids = df.user_id.unique()
np.random.seed(2020)
np.random.shuffle(ids)
train = int(0.6 * len(ids)) 
valid = int(0.2 * len(ids)) 

In [81]:
# split user_ids for three datasets
ids_train = np.array(ids[0:train])
ids_valid = np.array(ids[train:train + valid])
ids_test = np.array(ids[train + valid:])

In [85]:
train = df.loc[df['user_id'].isin(ids_train)]
valid = df.loc[df['user_id'].isin(ids_valid)]
test = df.loc[df['user_id'].isin(ids_test)]

In [86]:
train.shape[0]+valid.shape[0]+test.shape[0] == df.shape[0]

True

In [115]:
valid_train = []
valid_valid = []

In [116]:
for i in ids_valid:
    act = valid.loc[valid['user_id'] == i]
    length = int(act.shape[0]/2)
    to_train = act.iloc[:length]
    to_valid = act.iloc[length:]
    valid_train.append(to_train)
    valid_valid.append(to_valid)

In [117]:
test_train = []
test_test = []

In [118]:
for i in ids_test:
    act = test.loc[test['user_id'] == i]
    length = int(act.shape[0]/2)
    to_train = act.iloc[:length]
    to_test = act.iloc[length:]
    test_train.append(to_train)
    test_test .append(to_test)

In [119]:
valid_train = pd.concat(valid_train)
valid_valid = pd.concat(valid_valid)
test_train = pd.concat(test_train)
test_test = pd.concat(test_test)

In [124]:
train_df = [train, valid_train, test_train]
train = pd.concat(train_df)

In [125]:
train.shape[0]+valid_valid.shape[0]+test_test.shape[0]

1848958

In [127]:
train.to_csv("train.csv")

In [128]:
valid_valid.to_csv("valid.csv")

In [129]:
test_test.to_csv("test.csv")

In [130]:
train.to_json('train.json')

In [133]:
valid_valid.to_json('valid.json')

In [134]:
test_test.to_json('test.json')